# Start

## Methodology

**Caveat**
- In my first test (Stock_MA.ipynb), I use the first trading day of MONTH_ACT and first trading day of MONTH_PREV to define the the two ends of the 6m momentum. Here I use the Nth trading day instead. Therefore the result may vary
- If stock splits, the price momentum is unreasonable, I will check it and likely skip that stockthat year
- If using finviz
 - The BV is updated with Q4 results for some stocks
 - I can only use the whole SP500 instead of top 300
 - The momentum filter is not by rank, but by value like 10% increase 
- The SP500 components change by about 20 each year. To accurately apply the magic formula, I should use the sp500 components of that year for backtest, but I don't have it. Here I'm using the latest SP500 list instead. There's an information leak, since those in today's SP500 but not 5 years ago must have been performing well to get into the list in the past 5 years. So I'm picking up those stocks that I know increases a lot in value, and putting in my portfolio to test the performance. 
 - That said, I'm using top300 of sp500 which hopefully are more stable over time. And I have those strict criteria about momentum and PB. So the results should not be too far off


**Methodology (for backtest)** 
Here I get daily price history data and use the price of the action day (The Nth trading day of the action month) to compute price momentum. Also I use a latest full list of SP500, but take only the first N_TOP_BY_MKT_CAP stocks.

The follows are for the backtest. Prediction is similar except I only use one year's data and optimized parameters

Method: 
- Select the N_TOP_BY_MKT_CAP top stocks from SP500
- Starting from 2010, take actions on the Nth trading day of Feb (MONTH_ACT) of each year
- First choose the top 20% (TOP_BY_MMT = 0.2) of stocks ranked by 6 month price momentum (price of the Nth trading day of MONTH_ACT minus price of the Nth trading day of previous MONTH_PREV)
- Then choose the top N_STOCKS ranked by PB. PB = price of the Nth trading day of MONTH_ACT / book value of 2 quarters ago (previous Q3 if MONTH_ACT = Feb)
- On the Nth trading day of MONTH_ACT of each year, sell all stocks from the previous year with tax rate of 90% (TAX_FACTOR = (1 - 0.1)), and buy new stocks with the money by repeating the previous two steps  

**Parameters**
- Tried different N_TOP_BY_MKT_CAP, and 300 is better than the fullist
- Choose the top 40 gives good return among (30, 40, 50)
- Each year, the gain (after 10% lt tax) is better than SPY

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import akshare as ak  # https://github.com/akfamily/akshare

from stocklib import pick_stocks, get_quarter

pd.set_option('display.max_rows', 500)
%matplotlib inline

## Parameters

In [2]:
# Which stock was split in the past year; check back after finding those anomalous momentum
STOCK_SPLIT = ['IDONTKNOW']

## When to buy and sell
MONTH_ACT = 2  
YEAR_ACT = 2023
QUARTER_ACT = f'Q{MONTH_ACT // 3 + 1}'  # Dummy variable. If acting on Feb, the quarter is Q1

# Starting month to compute 6m momentum
MONTH_PREV = MONTH_ACT - 6 if MONTH_ACT - 6 > 0 else MONTH_ACT + 6  
# Starting year to compute 6m momentum
YEAR_PREV = YEAR_ACT if MONTH_ACT - 6 > 0 else YEAR_ACT - 1
# The quarter for the BV. Since the ER date differs, this should be 2 quarters ahead of QUARTER_ACT
# so that the BV is available for every stock. If acting in Q1, use previous Q3 
QUARTER_BV = f'Q{MONTH_ACT // 3 + 3}'  
# Trade on which trading day of the month. 
# The maximum of DOM varies by month; so be careful using a big value (> 20)
# Previously I use the first day of the MONTH_ACT for convienience. Now using DOM_TRADING allows me to
# compute on any day of the month. Note that the starting and end dates of 6m momentum is defined as 
# the Nth trading day of MONTH_PREV and that of MONTH_ACT, where N = DOM_TRADING. Therefore they do not
# always fall on the same calendar day of month
DOM_TRADING = 3 
DATE_CUTOFF = f"{YEAR_ACT}-{f'{MONTH_ACT}'.zfill(2)}-01"  # To filter data without recent data

## Filters to pick stocks and compute gains
N_TOP_BY_MKT_CAP = 300  # Choose from the top N of sp500
TOP_BY_MMT = 0.2  # The top fraction of stocks ranked by MMT
MMT_VAR = 'stock_price_mmt_6m'
TAX_FACTOR = (1 - 0.1)
N_STOCKS = 40  # Number of stocks to buy
TOTAL_CASH = 100000.0  # Total cash

## Parameters for both backtest and prediction
# Remove stocks with more than certain quarters with negatives BV. This is to reduce risks based on history.
# When apply the magic formula, only stocks with PB > 0 of the MONTH_ACT are considered
MAX_QUARTERS_NEG_PB = 40

## Backtest only parameters
YEAR_START = 2011  # backtest starting year
CASH_FOR_EACH_STOCK = 1000  # backtest cash for each stock
# Remove stocks with fewer than some quarters
MIN_YEARS_TEST = 3  # Keep stocks with >= MIN_YEARS_TEST years of data

In [3]:
file_name = {
    'fundamental': f'sp500_history_raw_{str(YEAR_ACT)}.csv',
    'price': f'sp500_history_price_raw_{str(YEAR_ACT)}.csv',
}

# Download SP500 component history
Downloading price takes about 0.5hr; PB taking much longer 

In [4]:
## Get fundamental data (seasonal) or price data (daily) 
# Download all SP500 instead of N_TOP_BY_MKT_CAP, since later filters will remove some

is_download = False

if is_download:
    is_from_scratch = False  # If starting from scratch and no stock data has been downloaded already
    to_download = 'price'  # 'fundamental' or 'price', download fundamental or price data
    
    min_row = {
        'fundamental': 3,
        'price': 190,
    }
    anom = {'Failed_PB': [], 'Short': [], 'Failed_P': []}

    # Get stock list (ordered by capital)
    df_sp500_list = pd.read_excel('sp500_fulllist_ranked.xlsx', engine='openpyxl', sheet_name=str(YEAR_ACT))
    stocks = df_sp500_list.loc[df_sp500_list['stock'] != 'GOOG', 'stock'].values.tolist()
    stocks.append('OHI')  # I like OHI

    if not is_from_scratch: 
        print('Read downloaded stocks from local file')
        df_stock_all = pd.read_csv(file_name[to_download])

    # If df_stock_all does not ecist, declare it
    try:
        df_stock_all       
    except NameError: 
        df_stock_all = pd.DataFrame()
    stock_downloaded = [] if df_stock_all.empty else df_stock_all.stock.unique()
    print(f'Downloaded {len(stock_downloaded)}, {stock_downloaded}')

    count = 0
    for stock_symbol in stocks:
        if stock_symbol in stock_downloaded:
            continue
        try:
            if to_download == 'fundamental':  # Download PB and PE
                try:  # If PE data can't be downloaded, record the stock
                    df_pb = ak.stock_us_fundamental(stock=stock_symbol, symbol="PB")
                except ValueError:
                    print(f'Download of {stock_symbol} PB failed. Skip for now')
                    anom['Failed_PB'].append(stock_symbol)
                    continue
                try:  # If PE data can't be downloaded, create and make it all nan
                    df_pe = ak.stock_us_fundamental(stock=stock_symbol, symbol="PE")
                except ValueError:
                    print(f'Download of {stock_symbol} PE failed. Using nan instead')
                    df_pe = df_pb.copy()
                    df_pe.columns = ['date', 'stock_price', 'ttm_net_eps', 'pe_ratio']
                    df_pe[['stock_price', 'ttm_net_eps', 'pe_ratio']] = np.nan
                df_stock = pd.merge(df_pe.drop('stock_price', axis=1), df_pb, on='date')
            elif to_download == 'price':  # Download price
                df_stock = ak.stock_us_daily(symbol=stock_symbol)
                df_stock = df_stock.reset_index()  # Set the date index to a column
            else:
                print('Wrong variable name')
        except IndexError:
            print(f'Failed for {stock_symbol}')
            anom['Failed_P'].append(stock_symbol)
            continue        
        df_stock['stock'] = stock_symbol
        df_stock_all = df_stock_all.append(df_stock)
        print(f"{(stock_symbol, df_stock.date.min(), df_stock.date.max(), len(df_stock))}")
        if len(df_stock) < min_row[to_download]:
            anom['Short'].append(stock_symbol)

        count += 1
        if count % 50 == 1:
            print('Saving...')
            df_stock_all.to_csv(file_name[to_download], index=False)    
    df_stock_all.to_csv(file_name[to_download], index=False)    

# Processe BV

## Filter by available length and action quarter

In [5]:
df_stock_all = pd.read_csv(file_name['fundamental'])
print(f'{df_stock_all.stock.nunique()} stocks downloaded')
df_stock_sub = get_quarter(df_stock_all)
df_stock_sub = df_stock_sub.drop(['DATE', 'dayofyear', ], axis=1).sort_values(['date', 'stock'])

# Convert $dollar to float
df_stock_sub['ttm_net_eps'] = df_stock_sub.ttm_net_eps.str[1:].replace('', np.nan).astype(float)
df_stock_sub['book_value_per_share'] = df_stock_sub.book_value_per_share.str[1:].replace('', np.nan).\
    str.replace(',', '').astype(float)
# Replace inf pe or pb to 0
df_stock_sub = df_stock_sub.replace(np.inf, 0)
# Remove stocks with over MAX_QUARTERS_NEG_PB quarters with neg equity (pb)
df_stock_sub_neg = df_stock_sub[df_stock_sub.price_to_book_ratio < 0].groupby('stock').size()
stocks_sub_neg = df_stock_sub_neg[df_stock_sub_neg >= MAX_QUARTERS_NEG_PB].index.values
df_stock_sub = df_stock_sub[~df_stock_sub.stock.isin(stocks_sub_neg)]

print(f'{df_stock_sub.stock.nunique()} stocks remained after filtering by neg BV')

# df_stock_sub.to_csv('sp500_history_filterd.csv')
# df_pb_history = pd.read_csv('sp500_history_filterd.csv', index_col=0)

## Get data for the quarter needed
df_pb_history = df_stock_sub
df_pb_quarter = df_pb_history[df_pb_history.index.str.endswith(QUARTER_BV)]\
    [['date', 'stock_price', 'book_value_per_share', 'stock', 'year']]\
    .reset_index(drop=True).rename({'year': 'year_prev', 'stock_price': 'stock_price_pb'}, axis=1)
print(f'{df_pb_quarter.stock.nunique()} stocks with the right quarter')

print(f'Stocks without {QUARTER_BV} data are {set(df_pb_history.stock.unique()) - set(df_pb_quarter.stock.unique())}')
# It's a new stock

498 stocks downloaded
493 stocks remained after filtering by neg BV
492 stocks with the right quarter
Stocks without Q3 data are {'GEHC'}


## Manually check and fill stocks with BV failure
I only filled data for the last quarter for prediction only

In [6]:
stocks_failed_pb = ['GM', 'HSY', 'CSGP', 'STT' ]  #  anom['Failed_P']
stocks_failed_pb_bv = [48.95, 15.26, 16.49, 69.7]   # get from https://www.macrotrends.net/

df_stocks_failed_pb = pd.DataFrame([
    [df_pb_quarter.date.max()] * len(stocks_failed_pb),
    [np.nan] * len(stocks_failed_pb),
    stocks_failed_pb_bv,
    stocks_failed_pb,
    [df_pb_quarter.year_prev.max()] * len(stocks_failed_pb)
]).T
df_stocks_failed_pb.columns = df_pb_quarter.columns

df_pb_quarter = df_pb_quarter.append(df_stocks_failed_pb)
print(f'{df_pb_quarter.stock.nunique()} stocks after manully filling in those failing in PB download')

496 stocks after manully filling in those failing in PB download


# Get price momentum and merge data

## Price momentum

In [7]:
df_p_history = pd.read_csv(file_name['price'])

In [8]:
df_p_history['date'] = pd.to_datetime(df_p_history['date'])
## Remove stocks that didn't last until the recent
dt_cutoff = pd.to_datetime(DATE_CUTOFF)
df_p_history['max_date'] = df_p_history.groupby('stock')['date'].transform(max)
df_p_history = (df_p_history[df_p_history['max_date'] >= dt_cutoff]).drop('max_date', axis=1)

## Get the starting and end dates to compute the momentum
# The starting date is the DOM_TRAIDING day of MONTH_ACT, and the end date is 
# the DOM_TRAIDING day of MONTH_PREV. The two may not be the same calendar DOM
df_p_history = df_p_history.sort_values(['stock', 'date'])
df_p_history['year'] = df_p_history.date.dt.year
df_p_history['month'] = df_p_history.date.dt.month
df_p_history['dom_trading'] = df_p_history.groupby(['stock', 'year', 'month'])['date'].rank()
df_p_history_prev = df_p_history[(df_p_history.month == MONTH_PREV) &
                                 (df_p_history.dom_trading == DOM_TRADING)]
df_p_history_curr = df_p_history[(df_p_history.month == MONTH_ACT) &
                                 (df_p_history.dom_trading == DOM_TRADING)]
## Use year_prev to join two datasets. 
# For 6month range, if MONTH_ACT is Jul-Dec, the year (year_prev) of MONTH_PREV is 
# the same year; otherwise its the previous year
df_p_history_curr['year_prev'] = df_p_history_curr.year if MONTH_ACT - 6 > 0 \
    else df_p_history_curr.year - 1
df_p_history_prev['year_prev'] = df_p_history_prev.year
cols = ['date', 'close', 'stock', 'year_prev']
df_p_history_mmt = pd.merge(df_p_history_prev[cols], 
                            df_p_history_curr[cols + ['year']],
                            on=['stock', 'year_prev'], 
                            suffixes=['_prev', '']
                           )

## Get the momentum
df_p_history_mmt['stock_price_mmt_6m'] = df_p_history_mmt['close'] / df_p_history_mmt['close_prev'] - 1

## Merge with BV

In [9]:
df_p_pb = pd.merge(df_p_history_mmt[['stock', 'year_prev', 'date', 'close', 
                                     'year', 'stock_price_mmt_6m']], 
                   df_pb_quarter, 
                   on=['stock', 'year_prev'], 
                   suffixes=['', '_pb'])\
            .rename({'close': 'stock_price'}, axis=1)\
            .drop(['year_prev'], axis=1)

df_p_pb['price_to_book_ratio'] = df_p_pb['stock_price'] / df_p_pb['book_value_per_share']

## Get top N by THIS YEAR'S market cap

In [10]:
# Choose only the top N companies of SP500 to start with
df_rank = pd.read_excel('sp500_fulllist_ranked.xlsx', engine='openpyxl', sheet_name=str(YEAR_ACT))
top_stocks = df_rank[df_rank['rank'] <= N_TOP_BY_MKT_CAP].stock.values
df_p_pb = df_p_pb[df_p_pb.stock.isin(top_stocks)]

print(f'{df_p_pb.stock.nunique()} of stocks after filtering by MKT CAP')

295 of stocks after filtering by MKT CAP


# Backtest

In [11]:
# Remove data with too short length and too late starting date
df_p_pb_size = df_p_pb.groupby('stock').size() 
df_p_pb_cut = df_p_pb[(df_p_pb.year >= YEAR_START) & 
                      (df_p_pb.stock.isin(df_p_pb_size[df_p_pb_size >= MIN_YEARS_TEST].index.values))]
print(f'{df_p_pb_cut.stock.nunique()} of stocks after filtering by length and starting date')

286 of stocks after filtering by length and starting date


In [50]:
top_by_mmt = TOP_BY_MMT
rsl_lt = pd.DataFrame()
# Number of stocks to choose by PB
for n_stocks in [30, 40, 50]:  
    # Invest method:
    # - At the beginning, get the top_by_mmt fraction of stocks ranked by MMT, 
    #   And choose the top n_stocks stocks ranked by PB
    # - After each holding period (PERIOD_HOLD), sell all stocks, and repurchase 
    #   with the initial methods
    cash_to_invest = CASH_FOR_EACH_STOCK * n_stocks
    cash_to_invest_prev = cash_to_invest

    # Run over time
    is_start = True
    period_cnt = 1
    for k, df_row in df_p_pb_cut.groupby('year'):    

        # Initialize
        if is_start:
            stocks_invested = pick_stocks(df_row, 
                              cash_to_invest=cash_to_invest, 
                              n_stocks=n_stocks, 
                              mmt_var=MMT_VAR, 
                              top_by_mmt=top_by_mmt
                             )
            is_start = False
            continue
        
        df_start = pd.merge(df_row[['stock', 'stock_price']], stocks_invested, on='stock', suffixes=('', '_bought'))
        values = (df_start['shares'] * df_start['stock_price']).sum()
        
        # Take action
        cash_to_invest_prev = cash_to_invest
        # Tax
        if values > cash_to_invest_prev:
            cash_to_invest = (values - cash_to_invest_prev) * TAX_FACTOR + cash_to_invest_prev
        else:
            cash_to_invest = values
        stocks_invested = pick_stocks(df_row, 
                                      cash_to_invest=cash_to_invest, 
                                      n_stocks=n_stocks, 
                                      mmt_var=MMT_VAR, 
                                      top_by_mmt=top_by_mmt
                                     )
        rsl_lt = rsl_lt.append(
            pd.DataFrame([k, round(values), 
                          round(values / (CASH_FOR_EACH_STOCK * n_stocks), 3), len(df_start),
                          n_stocks, MMT_VAR, 'lt_1', ], 
                         index=['year', 'value', 'return_overall', 'n_stocks_actual',
                                'n_stocks', 'MMT_VAR', 'method', ]).T) 

        period_cnt += 1
        

In [51]:
# Get SPY history
spy0 = ak.stock_us_daily(symbol="SPY", adjust="").reset_index()
spy = spy0.copy()
spy['year'] = spy.date.dt.year
spy['month'] = spy.date.dt.month
spy['dom_trading'] = spy.groupby(['year', 'month'])['date'].rank()

# Get the first day of MONTH_PREV and of MONTH_ACT, compute the momentum
spy_curr = spy[(spy.month == MONTH_ACT) & (spy.dom_trading == DOM_TRADING)]

spy_curr_cut = spy_curr[spy_curr.year >= YEAR_START]
spy_curr_cut['return_overall'] = (spy_curr_cut['close'] / 
                                  spy_curr_cut[spy_curr_cut.year == YEAR_START]['close'].\
                                  values[0]
                                 ).round(2)

In [52]:
df_compare = pd.merge(rsl_lt, spy_curr_cut[['year', 'return_overall']], 
                      on='year', 
                      suffixes=['', '_spy']).sort_values(['n_stocks', 'year'])
df_compare[df_compare.year.isin(
    [2020, 2021, 2022, 2023]
#     range(2011, 2024)
)][
    ['year', 'n_stocks_actual', 'n_stocks', 'return_overall', 'return_overall_spy']
].sort_values(['year', 'n_stocks'])

,year,n_stocks_actual,N_STOCKS,return_overall,return_overall_spy
24,2020,30,30,2.255,2.48
25,2020,40,40,2.805,2.48
26,2020,50,50,3.042,2.48
27,2021,30,30,3.059,2.88
28,2021,40,40,3.664,2.88
29,2021,50,50,4.032,2.88
30,2022,30,30,5.077,3.46
31,2022,40,40,5.6,3.46
32,2022,50,50,5.846,3.46
33,2023,30,30,5.259,3.14


In [54]:
# The higher the rank means the more years with best return
df_compare['return_overall'] = df_compare['return_overall'].astype(float)
df_compare['rank_n_stock'] = \
    df_compare.groupby('year')['return_overall'].rank()
df_compare.groupby('n_stocks')['rank_n_stock'].mean()

N_STOCKS
30    1.0
40    2.5
50    2.5
Name: rank_n_stock, dtype: float64

# Prediction
Apply the magic formula for a certain year

## Look for anoumalous mmt to detect split

In [14]:
df_row = df_p_pb[df_p_pb.year == YEAR_ACT]

In [15]:
## Check mmt anomalies manually
# df_row.sort_values('stock_price_mmt_6m')

# Check stocks with super low mmt on tradingview to see if they had a split
STOCK_SPLIT = ['TSLA', 'CPRT', ]  # And their momentum is low; just remove them
stocks_split = STOCK_SPLIT
df_row = df_row[~df_row.stock.isin(stocks_split)]

## Apply the magic formula to get candidate stocks

In [18]:
stocks_invested = pick_stocks(df_row, 
                              cash_to_invest=TOTAL_CASH,
                              n_stocks=N_STOCKS, 
                              mmt_var=MMT_VAR, 
                              top_by_mmt=TOP_BY_MMT,
                             )
stocks_invested['shares'] = stocks_invested['shares'].round()

In [21]:
stocks_invested.sort_values('price_to_book_ratio').reset_index(drop=True)

,stock,shares,stock_price,price_to_book_ratio,book_value_per_share,stock_price_mmt_6m_pct
0,STT,27.0,91.08,1.30674,69.7,28.0
1,LEN,24.0,104.81,1.31953,79.43,27.0
2,PCG,161.0,15.55,1.37489,11.31,40.0
3,DD,34.0,72.98,1.4215,51.34,26.0
4,JPM,18.0,141.09,1.43691,98.19,24.0
5,MS,25.0,99.43,1.65002,60.26,17.0
6,MPC,21.0,118.20,1.68978,69.95,28.0
7,DHI,25.0,99.47,1.72931,57.52,34.0
8,AFL,36.0,69.39,1.79256,38.71,18.0
9,HIG,33.0,74.77,1.8439,40.55,18.0


In [ ]:
# https://finviz.com/screener.ashx?v=150&f=idx_sp500,ta_perf_26w20o&ft=4&o=pb